In [2]:
import torch

In [13]:

# Create a tensor of given shape
x = torch.randn(2, 3, 2)

# Create a tensor of given shape
y = torch.randn(2, 2, 4)

z = x @ y

print(z.shape)
print(z)

# Calculate the dot product of x and y
z = torch.bmm(x, y)
print(z.shape)

print(z)


torch.Size([2, 3, 4])
tensor([[[-0.0408, -0.4732,  0.1724, -0.2304],
         [-0.1856,  0.1158, -0.0559,  0.7265],
         [-0.3722, -0.9696,  0.3330,  0.5162]],

        [[ 0.6053, -1.2011, -1.4210,  0.1257],
         [-0.3496,  1.8477,  2.6579,  1.4638],
         [ 0.7041, -1.1398, -1.2432,  0.4889]]])
torch.Size([2, 3, 4])
tensor([[[-0.0408, -0.4732,  0.1724, -0.2304],
         [-0.1856,  0.1158, -0.0559,  0.7265],
         [-0.3722, -0.9696,  0.3330,  0.5162]],

        [[ 0.6053, -1.2011, -1.4210,  0.1257],
         [-0.3496,  1.8477,  2.6579,  1.4638],
         [ 0.7041, -1.1398, -1.2432,  0.4889]]])


In [ ]:
# Create two tensors of the same shape (a, b, c)
a, b, c = 2, 3, 4  # Example dimensions
X = torch.randn(a, b, c)
Y = torch.randn(a, b, c)

# Compute Hadamard (element-wise) product
hadamard_product = X * Y

# Sum over all elements (generalized dot product)
GDT = hadamard_product.sum()

print("Generalized Dot Product (GDT):", GDT.item())  # Convert to scalar

GDT = torch.einsum('ijk,ijk->', X, Y)  # Summing over all elements
print("Generalized Dot Product (GDT):", GDT.item())

Generalized Dot Product (GDT): -1.056901454925537
Generalized Dot Product (GDT): -1.0569013357162476
